# Hosts: Asset trait introspection


## Setup

See "Hello OpenAssetIO" notebook for details on how to bootstrap OpenAssetIO. 

In [1]:
from resources import helpers

manager, context = helpers.bootstrap("resources/querying_entity_traits/openassetio_config.toml")

## Getting started

In the following examples we're going to ask the manager about the traits associated with an asset.

We've been given a URI by a colleague, which we need to turn into an `EntityReference` before we can use it to query the asset management system.

In [2]:
logo_ref = manager.createEntityReference("bal:///project_artwork/logos/openassetio")

helpers.display_result(repr(logo_ref))

> **Result:**
> `<openassetio.EntityReference bal:///project_artwork/logos/openassetio>`

Now we have an entity reference for our logo, we can use the API to learn more about it.

## The `entityTraits` method

The `entityTraits` methods allows a host to query the manager for the traits associated with a given entity reference. It has two access modes. `kRead` mode will return all the traits that the manager associates with an entity. `kWrite` mode returns the minimal set of traits required to publish to the given entity reference.

The `entityTraits` method is a required method that managers _must_ implement, so there is no need to perform a `hasCapability` check before using it.

The following examples illustrate usage of the `entityTraits` method.

### Available signatures

Like many OpenAssetIO API functions, there are multiple available signatures that provide a more convenient interface to the core batch-first callback-based signature:

In [3]:
import operator

from openassetio.hostApi import Manager
from openassetio.access import EntityTraitsAccess


# The core batch-first callback-based signature
callback_results = [None]

manager.entityTraits(
    [logo_ref], EntityTraitsAccess.kRead, context,
    lambda idx, result: operator.setitem(callback_results, idx, result),
    lambda idx, err: operator.setitem(callback_results, idx, err))

[callback_result] = callback_results


# Singular, exception-throwing
singular_result = manager.entityTraits(logo_ref, EntityTraitsAccess.kRead, context)

# Singular, success/error object return
singular_result_or_error = manager.entityTraits(
    logo_ref, EntityTraitsAccess.kRead, context, Manager.BatchElementErrorPolicyTag.kVariant)

# Batch, exception-throwing
[batch_result] = manager.entityTraits([logo_ref], EntityTraitsAccess.kRead, context)

# Batch, success/error object return.
[batch_result_or_error] = manager.entityTraits(
    [logo_ref], EntityTraitsAccess.kRead, context, Manager.BatchElementErrorPolicyTag.kVariant)


assert all(
    result == callback_result for result in
    (singular_result, singular_result_or_error, batch_result, batch_result_or_error))

helpers.display_result(callback_result)

> **Result:**
> `{'openassetio-mediacreation:identity.DisplayName', 'openassetio-mediacreation:usage.Entity', 'openassetio-mediacreation:twoDimensional.Image', 'openassetio-mediacreation:lifecycle.Version', 'openassetio-mediacreation:content.LocatableContent', 'openassetio-mediacreation:timeDomain.FrameRanged'}`

## Querying the full set of traits associated with an entity

OpenAssetIO does not try to fully define entity types. What it does define are traits that an entity satisfies. An entity is theoretically fully specified by some (large) subset of traits in the universe of all possible traits. A manager understands some subset of that theoretical complete subset of traits. This is the "trait set" of the entity, with respect to a given manager.

In order to find out what the trait set of an entity is, we can use the `entityTraits` API method.

In [4]:
from openassetio.access import EntityTraitsAccess

entity_trait_set = manager.entityTraits(logo_ref, EntityTraitsAccess.kRead, context)

helpers.display_result(entity_trait_set)

> **Result:**
> `{'openassetio-mediacreation:identity.DisplayName', 'openassetio-mediacreation:usage.Entity', 'openassetio-mediacreation:twoDimensional.Image', 'openassetio-mediacreation:lifecycle.Version', 'openassetio-mediacreation:content.LocatableContent', 'openassetio-mediacreation:timeDomain.FrameRanged'}`

`EntityTraitAccess.kRead` tells the manager the reason why you want the trait set. In this case we want to use the result in a read context, e.g. `resolve`ing properties, grouping entities of the same type in a UI, or determining the type of entity after a drag-and-drop of an entity reference into the application.

What happens when we `resolve` this trait set?

In [5]:
from openassetio.access import ResolveAccess

entity_data = manager.resolve(logo_ref, entity_trait_set, ResolveAccess.kRead, context)

helpers.display_result(entity_data)

> **Result:**
> `TraitsData({"openassetio-mediacreation:identity.DisplayName", "openassetio-mediacreation:content.LocatableContent", "openassetio-mediacreation:lifecycle.Version"})`

Note that not all traits have `resolve`able properties - many are simply used to aid classification (note the lack of `Entity` and `FrameRanged` traits). However, it is safe to `resolve` the full trait set - those that do not have properties are simply missing from the response.

As an aside, we can filter the trait set through `managementPolicy`, if we wish to know in advance which traits any given manager is capable of providing data for

In [6]:
from openassetio.access import PolicyAccess

[policy_data] = manager.managementPolicy([entity_trait_set], PolicyAccess.kRead, context)

helpers.display_result(policy_data)

> **Result:**
> `TraitsData({"openassetio-mediacreation:managementPolicy.Managed", "openassetio-mediacreation:lifecycle.Version", "openassetio-mediacreation:content.LocatableContent", "openassetio-mediacreation:identity.DisplayName"})`

Note how the trait set returned from `managementPolicy` (minus any `managementPolicy` traits) matches the trait set we successfully `resolve`d.  For example, this is missing the `FrameRangedTrait`, despite it being included in the result of `entityTraits`, meaning that the `FrameRangedTrait` is a quality of the entity, but properties of it cannot be resolved (perhaps due to some technical limitation of the manager).

In this way `managementPolicy` can be used to filter the trait set of an entity to only those traits that have `resolve`able properties. However, note that `mangementPolicy` does not take an entity reference argument, only trait set(s). The result of `managementPolicy` is therefore constant for any given manager, regardless of entity.

In practice, a well-behaved host will cherry-pick only those traits the host needs to `resolve`. That trait set should then be passed to `managementPolicy` to determine what the manager can actually provide. This process is independent of `entityTraits`. That is, the host should know at application startup which traits it's going to need, and so query `managementPolicy` ahead of time and store the result for use later. See relevant documentation for `managementPolicy` for more information.

## Querying the required set of traits for publishing an entity

By using the `kWrite` access mode of `entityTraits`, we can query the minimal trait set that _must_ be provided when publishing to a particular entity reference.

In most applications it is expected that the user will provide (via previous interaction with the manager) an entity reference that is suitable for a particular use-case. With this assumption, the host should not need to include additional data that is irrelevant for the use-case when publishing to an entity reference. That is, the host should not need to `resolve` data for a given entity reference for the sole purpose of immediately giving the data back again to the manager. 

This corresponds to an important philosophy of OpenAssetIO: when the user provides an entity reference, the host should not make assumptions about its provenance. It may be a reference to an existing entity, or a reference to a container, or a placeholder reference to an entity that doesn't exist yet, etc.

As such, `entityTraits` should rarely appear in publishing code. However, there are use cases where it is valuable.

### An empty trait set is a valid response

BAL has no restrictions on publishing to new entity references. So what happens if we just make up an entity reference, and try to get the trait set for it?

In [7]:
new_entity_ref = manager.createEntityReference("bal:///some/new/ref")

entity_trait_set = manager.entityTraits(new_entity_ref, EntityTraitsAccess.kWrite, context)

helpers.display_result(entity_trait_set)

> **Result:**
> `set()`

...an empty trait set is returned.

If the manager does not know the trait set, or it's not applicable, the returned trait set may be empty. For example, as illustrated here, the entity reference may point to a new entity with no type constraints. 

### Deciding whether an entity reference is appropriate

Let's say we are in a widget used for publishing 3D models. The user provides an entity reference they want to publish to. 

Since we deal with 3D models, we expect this entity reference to support publishing a trait set of `{EntityTrait.kId, GeometryTrait.kId, LocatableContentTrait.kId}`. But the user enters a reference to an image, not a 3D model...

In [8]:
from openassetio_mediacreation.traits.usage import EntityTrait
from openassetio_mediacreation.traits.content import LocatableContentTrait
from openassetio_mediacreation.traits.threeDimensional import GeometryTrait

entity_trait_set = manager.entityTraits(logo_ref, EntityTraitsAccess.kWrite, context)

helpers.display_result(entity_trait_set)

# Check if the minimal trait set required for publishing is satisfied by
# the trait set we're planning to publish.
is_entity_supported = entity_trait_set <= {EntityTrait.kId, GeometryTrait.kId, LocatableContentTrait.kId}

helpers.display_result(f"Does our widget support this entity? {is_entity_supported}")

> **Result:**
> `{'openassetio-mediacreation:twoDimensional.Image', 'openassetio-mediacreation:usage.Entity', 'openassetio-mediacreation:content.LocatableContent'}`

> **Result:**
> `Does our widget support this entity? False`

As expected, the trait set of the logo is not contained within our expected trait set, so our widget could flag an error to the user at this point.

A host could skip this check and try to publish regardless, reporting any error (from `preflight` or `register`) to the user. The `preflight` method is intended, in part, as a validation step, so this workflow is not unreasonable. However, doing this would likely provide a worse UX, since the widget could not be pre-emptively disabled.

#### Aside: the minimal trait set required for publishing 

In the previous example we see that the resulting trait set is smaller than that for `kRead`. This is because it is the minimal set of traits that the new entity _must_ possess when publishing to this entity reference.

Therefore, this trait set _must_ be imbued in the `TraitsData` given to `preflight` and `register` during the publishing process.

When registering new data, the full trait set defines what 'kind' of entity is being published, regardless of the specifics of what actual data is provided.

As a consequence, the minimal trait set _does not_ indicate whether a manager is capable of storing or dictating their properties (e.g. providing a file path to save to). See relevant documentation for `managementPolicy` for more information on how to determine that.

Again, many of these traits will not have properties associated with them at all. These must still be passed to `preflight`/`register` so that the manager knows the kind of entity you are publishing.

So what happens if we forget to include all these traits?

In [9]:
from openassetio.trait import TraitsData
from openassetio.access import PublishingAccess

data = TraitsData()
LocatableContentTrait(data).setLocation("file:///some/path")

entity_ref_or_error = manager.preflight(
    logo_ref, data, PublishingAccess.kWrite, context, manager.BatchElementErrorPolicyTag.kVariant)

helpers.display_result(entity_ref_or_error)

> **Result:**
> `BatchElementError(ErrorCode.kInvalidTraitSet, 'Publishing to this entity requires traits that are missing from the input')`

We get an `InvalidTraitSet` error if the provided trait set is not compatible, or sufficiently complete, for publishing to the entity reference.

## Errors when querying the trait set

### Entities that don't exist (yet)

For `kRead` access, the entity must exist:

In [10]:
future_ref = manager.createEntityReference("bal:///project_artwork/logos/new")

trait_set_or_error = manager.entityTraits(
    future_ref, EntityTraitsAccess.kRead, context, manager.BatchElementErrorPolicyTag.kVariant)

helpers.display_result(trait_set_or_error)

> **Result:**
> `BatchElementError(ErrorCode.kEntityResolutionError, 'Entity 'project_artwork/logos/new' not found')`

If the entity doesn't exist we get an `EntityResolutionError`.

On the other hand, this entity reference is fine for `kWrite`:

In [11]:
trait_set_or_error = manager.entityTraits(
    future_ref, EntityTraitsAccess.kWrite, context, manager.BatchElementErrorPolicyTag.kVariant)

helpers.display_result(trait_set_or_error)

> **Result:**
> `set()`

Here, once again, we get the minimal trait set required for publishing to this entity reference. Since the entity doesn't exist, BAL will allow any traits to be published to it, so the minimal trait set is empty.

### Read-only entities

Let's say that we have a reference to the current best/approved logo asset, and this entity is read-only to prevent artists overwriting it. What happens if we query `entityTraits` with a `kWrite` access mode, indicating we're planning to publish to it regardless?


In [12]:
approved_ref = manager.createEntityReference("bal:///project_artwork/logos/openassetio/approved")

trait_set_or_error = manager.entityTraits(
    approved_ref, EntityTraitsAccess.kWrite, context, manager.BatchElementErrorPolicyTag.kVariant)

helpers.display_result(trait_set_or_error)

> **Result:**
> `BatchElementError(ErrorCode.kEntityAccessError, 'Entity 'project_artwork/logos/openassetio/approved' is inaccessible for write')`

Since the entity is read-only, we get an `EntityAccessError`.